## Introduction

The Azure AI Agent Service Code Interpreter enables the LLM to generate Python code for tasks such as creating charts or performing complex data analyses based on user queries. It leverages natural language processing (NLP), sales data from an SQLite database, and user prompts to automate code generation. The LLM-generated Python code executes within a secure sandbox environment, utilizing a restricted subset of Python to ensure safe and controlled execution.


### Lab Exercise

In this lab, you'll enable the Code Interpreter to execute Python code generated by the LLM.

1. Open the main.py.

2. Uncomment the following lines by removing the "# " characters

```python
# INSTRUCTIONS_FILE = "instructions/instructions_code_interpreter.txt
# code_interpreter = CodeInterpreterTool()
# toolset.add(code_interpreter)
```

### Run the Agent App

Start a Conversation with the Agent

Try these questions:

1. Show sales by region as a pie chart

Once the task is complete, the pie chart image will be saved in the files folder. Open the folder in VS Code and click on the image file to view it.


2. Download the sales data

Once the task is complete, check the files folder to see the downloaded file.

3. Download as JSON

Once the task is complete, check the files folder to see the downloaded file.

4. Continue asking questions about Contoso sales data to see the Code Interpreter in action.